# Robustness Experiment Evaluation

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [27]:
file = "n1_RobustnessExperiment_2022-01-14_14-01-26.csv"


In [23]:
df_frame = pd.read_csv(file)
df_frame['makespan_std'] = df_frame['makespan'] - df_frame['makespan-optimum']
df_frame['makespan_uncertainty_std'] = df_frame['makespan_uncertainty'] - df_frame['makespan-optimum']
df_frame['makespan_uncertainty_baseline_delay'] = df_frame['makespan_uncertainty_std'] - df_frame['makespan_std']
list_ordering = {"RandomSolver": 0, "HillClimbing": 1, "TabuSearch": 2, "SimulatedAnnealing": 3, "GeneticAlgorithm": 4}
df_frame["solver"] = pd.Categorical(df_frame["solver"], categories=list_ordering)
df_frame.loc[~pd.notna(df_frame['robustness_measure']), 'robustness_measure'] = "No-RM"
df_frame.head(100)

,benchmark,solver,iterations,makespan-optimum,makespan,robustness,robustness_measure,uncertainty_percentage,makespan_uncertainty,robustness_uncertainty,makespan_std,makespan_uncertainty_std,makespan_uncertainty_baseline_delay
0,n1.mm/n110_1.mm,TabuSearch,5000,17,18,13.0,SF1,0.0,18,13.0,1,1,0
1,n1.mm/n110_1.mm,TabuSearch,5000,17,17,14.0,SF1,0.0,17,14.0,0,0,0
2,n1.mm/n110_1.mm,TabuSearch,5000,17,17,14.0,SF1,0.0,17,14.0,0,0,0
3,n1.mm/n110_1.mm,TabuSearch,5000,17,17,14.0,SF1,0.0,17,14.0,0,0,0
4,n1.mm/n110_1.mm,TabuSearch,5000,17,18,13.0,SF1,0.0,18,13.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,n1.mm/n110_1.mm,TabuSearch,5000,17,17,14.0,SF1,0.0,17,14.0,0,0,0
96,n1.mm/n110_1.mm,TabuSearch,5000,17,18,13.0,SF1,0.0,18,13.0,1,1,0
97,n1.mm/n110_1.mm,TabuSearch,5000,17,17,14.0,SF1,0.0,17,14.0,0,0,0
98,n1.mm/n110_1.mm,TabuSearch,5000,17,18,13.0,SF1,0.0,18,13.0,1,1,0


In [31]:
df_grouped = df_frame.groupby(['uncertainty_percentage', 'iterations', 'robustness_measure'])['makespan_uncertainty_baseline_delay'].describe()[['mean', 'std']]
df_grouped.head()

mean  std
uncertainty_percentage iterations robustness_measure           
0.0                    500        No-RM                0.0  0.0
                                  SF1                  0.0  0.0
                                  SF1_W1               0.0  0.0
                                  SF1_W9               0.0  0.0
                                  SF2                  0.0  0.0

In [25]:
# print(proactive_describe.to_latex(float_format="{:0.2f}".format))
#for solver in df_frame['solver'].unique():
group = df_grouped.reset_index()
 #   group = group[group['solver'] == solver]
group = group.set_index(['iterations', 'robustness_measure'])
group = group.pivot(columns='uncertainty_percentage')
print(group.to_latex(float_format="{:0.2f}".format))


\begin{tabular}{llrrrrrrrr}
\toprule
     & {} & \multicolumn{4}{l}{mean} & \multicolumn{4}{l}{std} \\
     & uncertainty\_percentage & 0.00 & 0.05 & 0.10 & 0.20 & 0.00 & 0.05 & 0.10 & 0.20 \\
iterations & robustness\_measure &      &      &      &      &      &      &      &      \\
\midrule
500  & No-RM & 1.84 & 2.85 & 3.71 & 5.30 & 2.27 & 2.78 & 3.04 & 3.42 \\
     & SF1 & 1.97 & 2.87 & 3.68 & 5.18 & 2.55 & 2.94 & 3.17 & 3.57 \\
     & SF1\_W1 & 1.84 & 2.73 & 3.55 & 5.02 & 2.42 & 2.78 & 3.03 & 3.35 \\
     & SF1\_W9 & 1.92 & 2.86 & 3.63 & 5.19 & 2.58 & 2.90 & 3.08 & 3.43 \\
     & SF2 & 1.59 & 2.44 & 3.24 & 4.72 & 2.15 & 2.55 & 2.85 & 3.19 \\
     & SF2\_W1 & 1.58 & 2.44 & 3.21 & 4.69 & 2.04 & 2.43 & 2.66 & 3.03 \\
     & SF2\_W9 & 1.62 & 2.51 & 3.32 & 4.83 & 2.15 & 2.64 & 2.96 & 3.33 \\
     & SF3 & 1.71 & 2.57 & 3.38 & 4.87 & 2.40 & 2.74 & 2.99 & 3.39 \\
     & SF3\_W1 & 1.89 & 2.76 & 3.55 & 5.04 & 2.37 & 2.68 & 2.90 & 3.21 \\
     & SF3\_W9 & 1.96 & 2.83 & 3.68 & 5.20 & 2.58 & 3.